In [1]:
# '''
# function ConnectButton(){
#     console.log("Connect pushed");
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
# }

# setInterval(ConnectButton,60000);
# '''

# from google.colab import drive
# from os import chdir

# drive.mount('/content/drive')
# project_path = '/content/drive/MyDrive/Gproject/o-linked-site-prediction-feature-augment'
# chdir(project_path)

In [2]:
import os
import numpy as np
import pandas as pd

# set seed for the reproducible result
SEED = 42

In [3]:
data_dir = './data/integrated_features' # we will get names from the augmented proteins
protein_names = [x.split('.')[0] for x in os.listdir(data_dir) if x.split('.')[1] == 'csv'] # get protein name list to be processed for building machine learning models
print('the number of initial proteins:', len(protein_names))
print(protein_names[:10])

the number of initial proteins: 104
['A2ABU4', 'A2AHJ4', 'A2AKB9', 'A2AQ25', 'E9Q1P8', 'E9Q5G3', 'O08537', 'O09061', 'O35303', 'O70263']


## hyper parameter optimization by K-fold cross-validation

In [4]:
import tensorflow as tf
import keras
from keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold

import time
from IPython.display import clear_output

from ml_models import *

epochs = 1000
from keras.callbacks import EarlyStopping
patience = 30
callbacks = [EarlyStopping(patience=patience, restore_best_weights=True, monitor='val_loss')]

test_size = 0.2


### set initial parameters

In [5]:
from functions import *

In [6]:
initial_params = default_params.copy()
initial_params.update({
    'window_size'    : 10
    })

print('initial parameters')
for key, value in initial_params.items():
    print(f'{key:<14} : {value}')

search_space = {
    'dnn_layers'    : [1, 2, 3, 4, 5],
    'dnn_neurons'   : [32, 64, 128, 256],
    'learning_rate' : [0.0001, 0.001, 0.01]
}

initial parameters
rnn_layers     : 1
rnn_neurons    : 64
dnn_layers     : 3
dnn_neurons    : 64
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


## set variables

In [7]:
model_type = 'MLP_UP'

# set continuous input features
x_cts = []
print('continuous features:')
display(dict(zip(range(len(x_cts)), x_cts)))

# set categorical input features
x_cat = ['residue']
print('categorical features:')
print(dict(zip(range(len(x_cat)), x_cat)))

# input features
x_var = x_cts + x_cat

# set continuos output feature
y_cts = []

# set categorical output feature
y_cat = ['positivity']

# output features
y_var = y_cts + y_cat

continuous features:


{}

categorical features:
{0: 'residue'}


# build amino acid sequence dataset

### model training with K-fold

In [8]:
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

model_update  = False

params = initial_params.copy()
MODELs = []
METRICs = []
METRIC_MEAN = []
model_id = 1
verbose = 0

data_x = []
data_y = []
for name in protein_names:
    data = pd.read_csv(f'./data/integrated_features/{name}.csv')
    ST_idx = np.where((data['residue'] == 'S') | (data['residue'] == 'T'))[0]
    
    # get X dataset
    x_onehot = get_onehots(data[x_var], columns = x_cat)
    x_features = list(x_onehot.columns)
    
    # get Y dataset
    y_onehot = get_onehots(data[y_var], columns = y_cat)
    y_labels = list(y_onehot.columns)
    
    for idx in ST_idx:
        window_x = np.array(get_window(x_onehot, idx, params['window_size']))
        window_x = window_x.reshape(-1)
        label_y  = np.array(y_onehot.iloc[idx])
        
        data_x.append(window_x)
        data_y.append(label_y)
        
data_x = np.array(data_x)
data_y = np.array(data_y)

for param_name, space in search_space.items():
    for point in space:
        clear_output(wait=True)
        display(METRIC_MEAN)
        params[param_name] = point
        for key, value in search_space.items():
            print(f'{key:<14} : {value}')

        model = MLP_CLS(data_x.shape[-1], data_y.shape[-1], params)  # I don't know why, but this row is helping producing the same training result of a neural network

        print('data x shape:', data_x.shape)
        print('data y shape:', data_y.shape)
        print('class y counts:', data_y.sum(0))
        print(f'class y ratio: {(data_y.sum(0)/len(data_y)).round(4)}')
        
        splitter = StratifiedShuffleSplit(n_splits = 1, test_size = test_size, random_state = SEED)
        train_idx, test_idx = list(splitter.split(data_x, data_y))[0]
        
        train_x = data_x[train_idx]
        train_y = data_y[train_idx]
        
        test_x = data_x[test_idx]
        test_y = data_y[test_idx]
        
        train_x, test_x = data_scaling(train_x, test_x)
        
        splitter_kf = KFold(n_splits = 5)
        for cv_idx, (train_idx_kf, test_idx_kf) in enumerate(splitter_kf.split(train_x, train_y)):
            train_x_kf, train_y_kf = train_x[train_idx_kf], train_y[train_idx_kf]
            test_x_kf, test_y_kf   = train_x[test_idx_kf],  train_y[test_idx_kf]
            
            train_x_kf, train_y_kf = upsample_data(train_x_kf, train_y_kf) # up-sample the training dataset
            
            model = MLP_CLS(data_x.shape[-1], data_y.shape[-1], params) 
            model_name = name_model(f'{model_type}_KFOLD', params)
            
            model_folder  = f'./models/{model_name}_{data_x.shape}'
            if not os.path.exists(model_folder):
                os.makedirs(model_folder)
            model_path    = f'{model_folder}/{cv_idx}.h5'
            metric_path   = f'{model_folder}/{cv_idx}.csv'
            
            
            if not os.path.exists(model_path) or model_update:
                time_start = time.time()
                history = model.fit(train_x_kf, train_y_kf, verbose=verbose, 
                                    epochs = 10000, callbacks = callbacks,
                                    validation_data = (test_x_kf, test_y_kf))
                time_end = time.time()
                training_time = round((time_end - time_start)/60, 3)
                
                model.save_weights(model_path)
                
                test_loss, accuracy, precision, recall, f1 = metrics_classification(model, test_x, test_y)
                model_metrics = {
                    'model_id' : model_id,
                    'cv_idx'   : cv_idx,
                    **{f'train_{x}': train_x.shape[x] for x in range(len(train_x.shape))},
                    'train_y'     : train_y.shape[-1],
                    'test_size'   : test_x.shape[0],
                    **params,
                    'regularizer_input' : params['regularizer']['input'],
                    'regularizer_hidden' : params['regularizer']['hidden'],
                    'regularizer_bias' : params['regularizer']['bias'],
                    'training_time': training_time,
                    'test_loss': test_loss,
                    'accuracy': accuracy,
                    **{f'precision_{x}': precision[x] for x in range(len(precision))},
                    **{f'recall_{x}'   : recall[x] for x in range(len(recall))},
                    **{f'f1_{x}'       : f1[x] for x in range(len(f1))}}
                
                model_metrics = pd.DataFrame([model_metrics]).drop(['activation', 'loss', 'metrics', 'optimizer_type', 'regularizer'], axis=1)
                model_metrics.to_csv(metric_path, index=False)
                
            else:
                model.load_weights(model_path)
                model_metrics = pd.read_csv(metric_path, header=0)
                
            print(f'f1 score: {model_metrics.f1_1[0]}')
            
            model_metrics['model_id'] = model_id
            METRICs.append(model_metrics)
            MODELs.append(model)
        
        METRIC_MEAN = pd.concat(METRICs).groupby('model_id').mean()
        f1_best = METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0].f1_1
        print(f'best f1 score: {f1_best}')
        model_id += 1
    if param_name in ['learning_rate']: # for float-type parameters
        best_value = float(METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0][param_name])
        params[param_name] = best_value
        search_space[param_name][search_space[param_name].index(best_value)] = f"{best_value}"
    else: # for int-type parameters
        best_value = int(METRIC_MEAN.sort_values('f1_1', ascending=False).iloc[0][param_name])
        params[param_name] = best_value
        search_space[param_name][search_space[param_name].index(best_value)] = f"{best_value}"

clear_output(wait=True)
display(METRIC_MEAN)
for key, value in search_space.items():
    print(f'{key:<14} : {value}')


,cv_idx,train_0,train_1,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,dnn_neurons,learning_rate,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
model_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,9222.0,420.0,2.0,2306.0,1.0,64.0,1.0,64.0,0.0010,...,NaN,0.3920,0.448702,80.418,96.942,6.764,82.252,33.024,88.988,11.218
2,2.0,9222.0,420.0,2.0,2306.0,1.0,64.0,2.0,64.0,0.0010,...,NaN,0.4258,0.447681,80.746,96.896,6.620,82.648,31.628,89.198,10.936
3,2.0,9222.0,420.0,2.0,2306.0,1.0,64.0,3.0,64.0,0.0010,...,NaN,0.3914,0.448122,80.392,96.890,6.524,82.272,31.860,88.976,10.820
4,2.0,9222.0,420.0,2.0,2306.0,1.0,64.0,4.0,64.0,0.0010,...,NaN,0.6952,0.445132,80.244,96.918,6.592,82.090,32.556,88.884,10.954
5,2.0,9222.0,420.0,2.0,2306.0,1.0,64.0,5.0,64.0,0.0010,...,NaN,0.6646,0.441334,80.548,96.916,6.684,82.414,32.326,89.068,11.062
6,2.0,9222.0,420.0,2.0,2306.0,1.0,64.0,1.0,32.0,0.0010,...,NaN,0.3496,0.445958,80.330,97.000,6.936,82.108,34.418,88.930,11.538
7,2.0,9222.0,420.0,2.0,2306.0,1.0,64.0,1.0,64.0,0.0010,...,NaN,0.3920,0.448702,80.418,96.942,6.764,82.252,33.024,88.988,11.218
8,2.0,9222.0,420.0,2.0,2306.0,1.0,64.0,1.0,128.0,0.0010,...,NaN,0.5840,0.450252,80.858,96.870,6.540,82.792,30.928,89.274,10.784
9,2.0,9222.0,420.0,2.0,2306.0,1.0,64.0,1.0,256.0,0.0010,...,NaN,0.7780,0.449549,80.918,96.862,6.506,82.866,30.696,89.312,10.726


dnn_layers     : ['1', 2, 3, 4, 5]
dnn_neurons    : ['32', 64, 128, 256]
learning_rate  : [0.0001, '0.001', 0.01]


In [9]:
print('data x shape: ', data_x.shape)
print('data y shape: ', data_y.shape)
print('train x shape:', train_x.shape)
print('test  x shape:', test_y.shape)

data x shape:  (11528, 420)
data y shape:  (11528, 2)
train x shape: (9222, 420)
test  x shape: (2306, 2)


In [10]:
print('optimal parameters')
for key, value in params.items():
    print(f'{key:<14} : {value}')

optimal parameters
rnn_layers     : 1
rnn_neurons    : 64
dnn_layers     : 1
dnn_neurons    : 32
activation     : softmax
loss           : categorical_crossentropy
metrics        : accuracy
optimizer_type : Adam
learning_rate  : 0.001
regularizer    : {'input': None, 'hidden': None, 'bias': None}
window_size    : 10


## evaluate model's general performance through Monte-Carlo cross-validation

In [11]:
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

model_update = False
            
MODELs = []
METRICs = []
METRIC_MEAN = []
verbose = 0
        
data_x = []
data_y = []

for name in protein_names:
    data = pd.read_csv(f'./data/integrated_features/{name}.csv')
    ST_idx = np.where((data['residue'] == 'S') | (data['residue'] == 'T'))[0]
    
    # get X dataset
    x_onehot = get_onehots(data[x_var], columns = x_cat)
    x_features = list(x_onehot.columns)
    
    # get Y dataset
    y_onehot = get_onehots(data[y_var], columns = y_cat)
    y_labels = list(y_onehot.columns)
    
    for idx in ST_idx:
        window_x = np.array(get_window(x_onehot, idx, params['window_size']))
        window_x = window_x.reshape(-1)
        label_y  = np.array(y_onehot.iloc[idx])
        
        data_x.append(window_x)
        data_y.append(label_y)
        
data_x = np.array(data_x)
data_y = np.array(data_y)

print('data x shape:', data_x.shape)
print('data y shape:', data_y.shape)
print('class y counts:', data_y.sum(0))
print(f'class y ratio: {(data_y.sum(0)/len(data_y)).round(4)}')

splitter = StratifiedShuffleSplit(n_splits = 5, test_size = test_size, random_state = SEED)
for cv_idx, (train_idx, test_idx) in enumerate(splitter.split(data_x, data_y)):
    train_x, train_y = data_x[train_idx], data_y[train_idx]
    test_x , test_y  = data_x[test_idx],  data_y[test_idx]
    
    train_x, train_y = upsample_data(train_x, train_y) # up-sample the training dataset
    
    train_x, test_x = data_scaling(train_x, test_x)
    
    model = MLP_CLS(data_x.shape[-1], data_y.shape[-1], params)
    model_name = name_model(f'{model_type}', params)
    
    
    model_folder  = f'./models/{model_name}_{data_x.shape}'
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)
    model_path    = f'{model_folder}/{cv_idx}.h5'
    metric_path   = f'{model_folder}/{cv_idx}.csv'
    
    
    if not os.path.exists(model_path) or model_update:
        time_start = time.time()
        history = model.fit(train_x, train_y, verbose=verbose, 
                            epochs = 10000, callbacks = callbacks,
                            validation_split = test_size/(1-test_size))
        time_end = time.time()
        training_time = round((time_end - time_start)/60, 3)
        
        model.save_weights(model_path)
        
        test_loss, accuracy, precision, recall, f1 = metrics_classification(model, test_x, test_y)
        model_metrics = {
            'model_id' : model_id,
            'cv_idx'   : cv_idx,
            **{f'train_{x}': train_x.shape[x] for x in range(len(train_x.shape))},
            'train_y'     : train_y.shape[-1],
            'test_size'   : test_x.shape[0],
            **params,
            'regularizer_input' : params['regularizer']['input'],
            'regularizer_hidden' : params['regularizer']['hidden'],
            'regularizer_bias' : params['regularizer']['bias'],
            'training_time': training_time,
            'test_loss': test_loss,
            'accuracy': accuracy,
            **{f'precision_{x}': precision[x] for x in range(len(precision))},
            **{f'recall_{x}'   : recall[x] for x in range(len(recall))},
            **{f'f1_{x}'       : f1[x] for x in range(len(f1))}}
        model_metrics = pd.DataFrame([model_metrics]).drop(['activation', 'loss', 'metrics', 'optimizer_type', 'regularizer'], axis=1)
        model_metrics.to_csv(metric_path, index=False)
        
    else:
        model.load_weights(model_path)
        model_metrics = pd.read_csv(metric_path, header=0)
        
    print(f'f1 score: {model_metrics.f1_1[0]}')
    
    METRICs.append(model_metrics)
    MODELs.append(model)

METRICs = pd.concat(METRICs)
METRIC_MEAN = METRICs.groupby('model_id').mean()
METRIC_STD = METRICs.groupby('model_id').std()

data x shape: (11528, 420)
data y shape: (11528, 2)
class y counts: [11100   428]
class y ratio: [0.9629 0.0371]
f1 score: 11.18


c:\Users\honsu\OneDrive\Github\glycosylation\o-linked-site-prediction-feature-augment\functions.py:103: RuntimeWarning: invalid value encountered in divide
  train_data_sc = (train_data - x_min) / (x_max - x_min)
c:\Users\honsu\OneDrive\Github\glycosylation\o-linked-site-prediction-feature-augment\functions.py:104: RuntimeWarning: invalid value encountered in divide
  test_data_sc  = (test_data - x_min)  / (x_max - x_min)
c:\Users\honsu\OneDrive\Github\glycosylation\o-linked-site-prediction-feature-augment\functions.py:103: RuntimeWarning: invalid value encountered in divide
  train_data_sc = (train_data - x_min) / (x_max - x_min)
c:\Users\honsu\OneDrive\Github\glycosylation\o-linked-site-prediction-feature-augment\functions.py:104: RuntimeWarning: invalid value encountered in divide
  test_data_sc  = (test_data - x_min)  / (x_max - x_min)


f1 score: 12.590000000000002
f1 score: 14.09


c:\Users\honsu\OneDrive\Github\glycosylation\o-linked-site-prediction-feature-augment\functions.py:103: RuntimeWarning: invalid value encountered in divide
  train_data_sc = (train_data - x_min) / (x_max - x_min)
c:\Users\honsu\OneDrive\Github\glycosylation\o-linked-site-prediction-feature-augment\functions.py:104: RuntimeWarning: invalid value encountered in divide
  test_data_sc  = (test_data - x_min)  / (x_max - x_min)
c:\Users\honsu\OneDrive\Github\glycosylation\o-linked-site-prediction-feature-augment\functions.py:103: RuntimeWarning: invalid value encountered in divide
  train_data_sc = (train_data - x_min) / (x_max - x_min)
c:\Users\honsu\OneDrive\Github\glycosylation\o-linked-site-prediction-feature-augment\functions.py:104: RuntimeWarning: invalid value encountered in divide
  test_data_sc  = (test_data - x_min)  / (x_max - x_min)


f1 score: 13.89
f1 score: 11.94


c:\Users\honsu\OneDrive\Github\glycosylation\o-linked-site-prediction-feature-augment\functions.py:103: RuntimeWarning: invalid value encountered in divide
  train_data_sc = (train_data - x_min) / (x_max - x_min)
c:\Users\honsu\OneDrive\Github\glycosylation\o-linked-site-prediction-feature-augment\functions.py:104: RuntimeWarning: invalid value encountered in divide
  test_data_sc  = (test_data - x_min)  / (x_max - x_min)


In [12]:
METRICs

,model_id,cv_idx,train_0,train_1,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,dnn_neurons,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
0,13,0,17760,420,2,2306,1,64,1,32,...,NaN,0.921,0.526099,75.89,97.11,6.48,77.25,40.70,86.05,11.18
0,13,1,17760,420,2,2306,1,64,1,32,...,NaN,0.858,0.588330,73.50,97.52,7.18,74.37,51.16,84.39,12.59
0,13,2,17760,420,2,2306,1,64,1,32,...,NaN,1.232,0.511851,75.67,97.70,8.11,76.53,53.49,85.83,14.09
0,13,3,17760,420,2,2306,1,64,1,32,...,NaN,0.646,0.546221,73.11,97.85,7.89,73.69,58.14,84.07,13.89
0,13,4,17760,420,2,2306,1,64,1,32,...,NaN,0.879,0.500908,76.32,97.23,6.93,77.61,43.02,86.32,11.94


In [13]:
METRIC_MEAN

,cv_idx,train_0,train_1,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,dnn_neurons,learning_rate,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
model_id,,,,,,,,,,,,,,,,,,,,,
13,2.0,17760.0,420.0,2.0,2306.0,1.0,64.0,1.0,32.0,0.001,...,NaN,0.9072,0.534682,74.898,97.482,7.318,75.89,49.302,85.332,12.738


In [14]:
METRIC_STD

,cv_idx,train_0,train_1,train_y,test_size,rnn_layers,rnn_neurons,dnn_layers,dnn_neurons,learning_rate,...,regularizer_bias,training_time,test_loss,accuracy,precision_0,precision_1,recall_0,recall_1,f1_0,f1_1
model_id,,,,,,,,,,,,,,,,,,,,,
13,1.581139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.210432,0.034449,1.479314,0.310757,0.6757,1.758408,7.289713,1.027093,1.249108
